In [1]:
!pip install transformers torch


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import tkinter as tk
from tkinter import scrolledtext
import random
import re
import wikipedia
import requests

# Path to the model directory
model_path = r"C:\Users\Vedashree\OneDrive\Desktop\DialoGPT-medium\DialoGPT-master"

# Load pre-trained model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Move model to GPU if available, otherwise keep it on CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Initialize conversation history to store messages
conversation_history = []

# Create the main window
root = tk.Tk()
root.title("Comprehensive Chatbot")
root.geometry("600x650")
root.configure(bg="#2C3E50")  # Dark blue background

# Create a frame for the chat history
frame = tk.Frame(root, bg="#34495E", padx=10, pady=10)
frame.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

# Create a ScrolledText widget for displaying conversation history
chat_history = scrolledtext.ScrolledText(frame, state='disabled', wrap=tk.WORD, height=20, width=58, bg="#ECF0F1", fg="#2C3E50", font=("Arial", 12))
chat_history.pack(fill=tk.BOTH, expand=True)

# Create a frame for user input
input_frame = tk.Frame(root, bg="#2C3E50")
input_frame.pack(pady=10, padx=10, fill=tk.X)

# Create an Entry widget for user input
user_input = tk.Entry(input_frame, width=40, font=("Arial", 12), bg="#ECF0F1", fg="#2C3E50")
user_input.pack(side=tk.LEFT, padx=10, pady=10, fill=tk.X, expand=True)

def analyze_question_type(query):
    """Analyze the type of question being asked."""
    query_lower = query.lower()
    
    # Factual questions
    if any(phrase in query_lower for phrase in ["what is", "who is", "where is", "when did", "how does", "why is", "tell me about"]):
        return "factual"
    
    # Step-by-step instructions
    if any(phrase in query_lower for phrase in ["how to", "steps to", "guide", "tutorial", "instructions"]):
        return "procedural"
    
    # Opinions or subjective questions
    if any(phrase in query_lower for phrase in ["opinion", "think about", "perspective", "viewpoint", "better", "should i"]):
        return "subjective"
    
    # Comparison questions
    if any(phrase in query_lower for phrase in ["versus", "vs", "compare", "difference between", "similarities"]):
        return "comparison"
    
    # Definition questions
    if any(phrase in query_lower for phrase in ["define", "meaning of", "definition"]):
        return "definition"
    
    return "conversation"

def analyze_complexity(query):
    """Analyze the complexity of the query to adapt response detail."""
    # Count words as a basic complexity measure
    word_count = len(query.split())
    
    if word_count > 12:
        return "high"
    elif word_count > 6:
        return "medium"
    else:
        return "low"

def analyze_and_generate_response(user_message):
    """Analyzes the question and generates an appropriate response."""
    user_message = user_message.strip()
    user_message_lower = user_message.lower()
    
    # Detect question type and complexity
    question_type = analyze_question_type(user_message)
    complexity = analyze_complexity(user_message)
    
    # Handle greetings and common conversation starters
    if re.match(r"^(hi|hello|hey)$", user_message_lower):
        return random.choice([
            "Hi there! How can I assist you today?", 
            "Hello! I'm here to help. What would you like to know?",
            "Hey! I'm ready to answer any questions you might have."
        ])
    
    if "how are you" in user_message_lower:
        return "I'm doing well, thanks for asking! How can I help you today?"
    
    # Handle factual questions with Wikipedia
    if question_type == "factual":
        wiki_response = fetch_wikipedia_summary(user_message)
        if "couldn't find any information" not in wiki_response:
            return format_response(wiki_response, complexity)
    
    # Handle specific queries like weather or news
    if "weather" in user_message_lower:
        return fetch_weather_info(user_message)
    elif "news" in user_message_lower:
        return fetch_news_info()
    
    # Handle subjective questions with balanced views
    if question_type == "subjective":
        return generate_balanced_response(user_message)
    
    # Handle procedural questions with step-by-step instructions
    if question_type == "procedural":
        return generate_procedural_response(user_message)
    
    # Handle comparison questions
    if question_type == "comparison":
        return generate_comparison_response(user_message)
    
    # Handle definition questions
    if question_type == "definition":
        return generate_definition_response(user_message)
    
    # Fallback to model-based conversation
    return generate_response_from_model(user_message)

def format_response(content, complexity):
    """Format the response based on complexity."""
    # Adapt language complexity
    if complexity == "high":
        content = content  # Keep detailed explanation
    elif complexity == "medium":
        # Add an explanatory note for medium complexity
        if len(content.split()) > 30:
            content = content + "\n\nDoes that help answer your question? Let me know if you need more details."
    else:
        # Simplify for low complexity by keeping first couple sentences
        sentences = content.split('.')
        if len(sentences) > 3:
            content = '.'.join(sentences[:3]) + '.'
            content += "\n\nI can provide more details if you'd like."
    
    return content

def generate_balanced_response(query):
    """Generate a response for subjective questions with multiple perspectives."""
    # Extract the main topic from the query
    topic = re.sub(r'^(what|who|where|when|why|how|should|could|would|is|are|do|does|did|can|could) ', '', query.lower())
    topic = re.sub(r'\?$', '', topic)
    
    response = f"When it comes to {topic}, there are several perspectives to consider:\n\n"
    response += "On one hand, some might argue that this approach has benefits such as improved efficiency and broader accessibility.\n\n"
    response += "On the other hand, others might point out potential concerns including long-term sustainability and ethical considerations.\n\n"
    response += "A balanced view would acknowledge both sides while considering your specific context and priorities."
    
    return response

def generate_procedural_response(query):
    """Generate step-by-step instructions for procedural questions."""
    # Extract the topic from the query
    match = re.search(r"how to (.*?)(?:\?|$)", query.lower())
    topic = match.group(1) if match else query
    
    steps = [
        f"First, understand the basics of {topic} by gathering necessary information.",
        f"Next, prepare all required tools and materials for {topic}.",
        f"Then, start with the most fundamental step of {topic}, which typically involves initial setup.",
        f"As you progress, pay attention to details and make adjustments as needed.",
        f"Finally, review your work and make any necessary refinements."
    ]
    
    response = f"Here's a step-by-step guide for {topic}:\n\n"
    for i, step in enumerate(steps, 1):
        response += f"{i}. {step}\n"
    
    response += f"\nIf you run into any issues with this approach, there are alternative methods you could try. Would you like me to explain any of these steps in more detail?"
    
    return response

def generate_comparison_response(query):
    """Generate comparison between items/concepts."""
    # Try to extract items being compared
    items = re.findall(r"between (.*?) and (.*?)(?:\?|$)", query.lower())
    if not items:
        return generate_response_from_model(query)
    
    item1, item2 = items[0]
    
    response = f"When comparing {item1} and {item2}:\n\n"
    response += f"Similarities:\n- Both serve valuable purposes in their respective contexts\n- Both have developed over time with increasing sophistication\n\n"
    response += f"Key differences:\n- {item1.capitalize()} tends to be more focused on specific applications\n- {item2.capitalize()} often offers different advantages in terms of flexibility and scope\n\n"
    response += f"The choice between {item1} and {item2} ultimately depends on your specific needs and circumstances."
    
    return response

def generate_definition_response(query):
    """Generate definition for terms."""
    # Extract the term to define
    match = re.search(r"(?:define|what is|meaning of) (.*?)(?:\?|$)", query.lower())
    term = match.group(1) if match else query
    
    # Try Wikipedia first
    wiki_response = fetch_wikipedia_summary(term)
    if "couldn't find any information" not in wiki_response:
        return f"{term.capitalize()} refers to {wiki_response}"
    
    # Fallback to generated response
    response = f"{term.capitalize()} typically refers to a concept that "
    response += random.choice([
        "involves interconnected elements working together to achieve a specific purpose.",
        "describes a process or method used in various contexts.",
        "represents an important principle or idea in its field."
    ])
    
    return response

def fetch_wikipedia_summary(query):
    """Fetch a comprehensive summary from Wikipedia API."""
    # Extract the main subject from the query
    search_terms = re.sub(r'^(what is|who is|tell me about|how does|when is|where is|why is|define|meaning of) ', '', query.lower())
    search_terms = re.sub(r'\?$', '', search_terms)
    
    try:
        # Search Wikipedia for the query
        search_results = wikipedia.search(search_terms, results=3)
        if not search_results:
            return "I'm sorry, I couldn't find any information on that topic. You might want to check a specialized resource or rephrase your question."
        
        # Try to get the most relevant page
        try:
            page = wikipedia.page(search_results[0])
        except wikipedia.exceptions.DisambiguationError as e:
            # If disambiguation page, take the first option
            page = wikipedia.page(e.options[0])
        
        # Get a longer summary (3-4 sentences)
        summary = wikipedia.summary(page.title, sentences=4)
        
        # Add a source attribution
        summary += f"\n\nThis information comes from Wikipedia's article on '{page.title}'."
        
        return summary
    
    except wikipedia.exceptions.PageError:
        return "I'm sorry, I couldn't find specific information on that. Consider checking specialized resources for more details."
    except Exception as e:
        return f"I encountered an issue while searching for information. You might want to try rephrasing your question or consult a specialized resource."

def fetch_weather_info(query):
    """Fetch weather info based on user input with better city extraction."""
    # More robust city extraction
    city_match = re.search(r'weather (?:in|for|at) ([A-Za-z\s]+)', query)
    city = city_match.group(1).strip() if city_match else extract_city_from_query(query)
    
    if city:
        return get_weather_data(city)
    return "Please specify a city for the weather information. For example, 'What's the weather in New York?'"

def get_weather_data(city):
    """Fetch weather data using an API (replace with an actual API call)."""
    # This is a placeholder. In a real implementation, you would use a weather API
    weather_conditions = ["sunny", "partly cloudy", "overcast", "rainy", "stormy", "snowy"]
    temperatures = range(0, 35)
    
    condition = random.choice(weather_conditions)
    temp = random.choice(temperatures)
    
    response = f"The current weather in {city} is {condition} with a temperature of {temp}°C."
    
    # Add a practical suggestion based on weather
    if condition == "sunny" and temp > 25:
        response += " It's quite warm, so remember to stay hydrated if you're going outside."
    elif condition == "rainy":
        response += " Don't forget your umbrella if you're heading out!"
    elif condition == "snowy" or temp < 5:
        response += " Bundle up, it's cold outside!"
    
    response += "\n\nNote: This is simulated weather data. For accurate forecasts, please check a weather service."
    
    return response

def fetch_news_info():
    """Fetch top news headlines with categorization."""
    # This is a placeholder. In a real implementation, you would use a news API
    headlines = [
        "New Study Reveals Benefits of Regular Exercise",
        "Tech Company Unveils Latest Smartphone Model",
        "Scientists Make Breakthrough in Renewable Energy",
        "Global Leaders Meet to Discuss Climate Change",
        "Stock Markets Show Strong Performance This Week",
        "New Film Receives Critical Acclaim at Festival"
    ]
    
    response = "Here are some recent headlines:\n\n"
    for headline in headlines:
        response += f"- {headline}\n"
    
    response += "\nNote: These are simulated headlines. For current news, please check a news service."
    
    return response

def extract_city_from_query(query):
    """Extract city name from the user's query with an expanded list."""
    # Expanded city list
    cities = ["New York", "London", "Paris", "Tokyo", "Beijing", "Sydney", "Moscow", 
              "Cairo", "Mumbai", "Rio de Janeiro", "Toronto", "Berlin", "Rome", "Madrid"]
    
    query_lower = query.lower()
    for city in cities:
        if city.lower() in query_lower:
            return city
    return None

def generate_response_from_model(user_message):
    """Generate a response using the DialoGPT model."""
    global conversation_history
    conversation_history.append(f"You: {user_message}")
    
    # Limit conversation history to prevent token overflow
    if len(conversation_history) > 10:
        conversation_history = conversation_history[-10:]
    
    input_text = " ".join(conversation_history[-6:])
    input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt').to(device)
    
    # Generate response with better parameters for more coherent output
    output_ids = model.generate(
        input_ids, 
        max_length=150, 
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=torch.ones(input_ids.shape, device=input_ids.device),
        top_p=0.92, 
        top_k=50, 
        temperature=0.7, 
        no_repeat_ngram_size=3,
        do_sample=True, 
        num_return_sequences=1
    )
    
    bot_response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    bot_response = bot_response[len(input_text):].strip()
    
    # Clean up the response if it's empty or too short
    if not bot_response or len(bot_response) < 5:
         bot_response = "I understand your message. Could you provide more details so I can give you a better answer?"
    
    # Format the response to reflect the guidelines given in the prompts
    bot_response = format_model_response(bot_response)
    
    return bot_response

def format_model_response(response):
    """Format model response to align with the assistant's guidelines."""
    # Clean up any artifacts in the generated text
    response = re.sub(r'You: .*?Bot: ', '', response)
    response = re.sub(r'Bot: ', '', response)
    
    # Format response with line breaks for readability
    if len(response.split()) > 30:
        sentences = response.split('. ')
        if len(sentences) > 3:
            formatted_response = '. '.join(sentences[:3]) + '.'
            remaining = '. '.join(sentences[3:])
            if remaining:
                formatted_response += '\n\n' + remaining
            response = formatted_response
    
    return response

def send_message(event=None):
    """Process user input and generate a response."""
    message = user_input.get()
    if message.strip() == "":
        return
    
    # Clear the input field
    user_input.delete(0, tk.END)
    
    # Display user message
    chat_history.configure(state='normal')
    chat_history.insert(tk.END, f"You: {message}\n\n", "user_msg")
    chat_history.tag_configure("user_msg", foreground="#2C3E50", font=("Arial", 12, "bold"))
    
    # Generate response
    bot_message = analyze_and_generate_response(message)
    
    # Display bot message with slight delay for natural feel
    root.after(300, lambda: display_bot_response(bot_message))
    
    # Add to conversation history
    conversation_history.append(f"Bot: {bot_message}")

def display_bot_response(message):
    """Display the bot's response in the chat history."""
    chat_history.configure(state='normal')
    chat_history.insert(tk.END, "Assistant: ", "bot_tag")
    chat_history.tag_configure("bot_tag", foreground="#2980B9", font=("Arial", 12, "bold"))
    chat_history.insert(tk.END, f"{message}\n\n", "bot_msg")
    chat_history.tag_configure("bot_msg", foreground="#2980B9", font=("Arial", 12))
    chat_history.see(tk.END)
    chat_history.configure(state='disabled')

# Bind the send_message function to the Return key
user_input.bind('<Return>', send_message)

# Create a Send button
send_button = tk.Button(input_frame, text="Send", command=send_message, bg="#3498DB", fg="white", font=("Arial", 12, "bold"), padx=10)
send_button.pack(side=tk.RIGHT, padx=10, pady=10)

# Create a function to clear the chat history
def clear_chat():
    global conversation_history
    conversation_history = []
    chat_history.configure(state='normal')
    chat_history.delete(1.0, tk.END)
    chat_history.configure(state='disabled')

# Create a Clear button
clear_button = tk.Button(root, text="Clear Chat", command=clear_chat, bg="#E74C3C", fg="white", font=("Arial", 10))
clear_button.pack(pady=5)

# Add an info message at the start
chat_history.configure(state='normal')
welcome_message = "Welcome! I'm an AI assistant designed to help answer your questions across various domains including science, history, technology, arts, and everyday advice. Feel free to ask me anything, and I'll do my best to assist you.\n\n"
chat_history.insert(tk.END, welcome_message, "info_msg")
chat_history.tag_configure("info_msg", foreground="#7F8C8D", font=("Arial", 11, "italic"))
chat_history.configure(state='disabled')

# Start the main loop
root.mainloop()